## Hands-on session 2.3 - CRM basics with ReMKiT1D

In this session we cover the basics of CRM construction for use in ReMKiT1D models. The example is focused on a small set of equations, in order to demonstrate the general concepts. 

Demonstrated concepts:

- Setting species and associating variables to them 
- Simple and derived transitions 
- Constructing CRM modelbound data from transitions
- Adding term generators that use CRM modelbound data
- The extractor manipulator

A number of CRM concepts are beyond this workshop. These include:

- Kinetic features - Boltzmann term generators 
- Built-in Janev hydrogen transitions 
- Using external databases such as AMJUEL to construct polynomial fits 
- Other kinds of transitions (detailed balance etc.)

In [ ]:
from RMK_support import RKWrapper ,Grid, Node, treeDerivation
import RMK_support.simple_containers as sc
import RMK_support.IO_support as io
import RMK_support.dashboard_support as ds 
import RMK_support.crm_support as crm

import numpy as np
import holoviews as hv 
import panel as pn
import matplotlib.pyplot as plt

### Wrapper initialization

In [ ]:
rk = RKWrapper()

### Global parameters for writing the files

In [ ]:
rk.jsonFilepath = "./config.json" # Default value
hdf5Filepath = "./RMKOutput/day_2_3/"
rk.setHDF5Path(hdf5Filepath) 

### Grid initialization

We initialize 0D grid as in the previous session

In [ ]:
rk.grid = Grid(np.zeros(1))

### A simplified Collisional Radiative Model 

Let's write a simplified nonlinear time-dependent CRM. 

Let's assume we're working with electrons, singly-charged ions, a ground state and an excited neutral state. This implies that we are tracking the following densities:

$$n_e,n_i,n_1,n_2$$

Let's then say we have the following reactions in the system:

- Electron-impact ionization of both states 

$$ e + b \rightarrow i + e + e,\quad b=1,2$$  

- The direct inverse of the above reaction - three-body recombination

$$ i + e + e \rightarrow b + e,\quad b=1,2$$  

- Electron impact excitation from state 1 to state 2

$$ e + 1 \rightarrow e + 2$$  

- Radiative de-excitation (spontaneous emission) from state 2 to state 1 

$$ n_2 \rightarrow n_1 + h\nu$$ 
where $h\nu$ signifies a photon with the transition energy. We will assume that any energy losses are recuperated through some heating of the electrons, maintaining a constant electron temperature. This is assumption can be relaxed, but required the evolution of electron energy and unduly complicated this system.

The system described above can be represented with the following system of nonlinear ODEs

$$ \frac{d n_1}{dt} = -K_{12}n_en_1 - K_1^{ion}n_en_1 + A_{21}n_2 + K_1^{rec}n_e^2n_i $$

$$ \frac{d n_2}{dt} = K_{12}n_en_1 - K_2^{ion}n_en_2 - A_{21}n_2 + K_2^{rec}n_e^2n_i $$
$$ \frac{d n_e}{dt} = \frac{d n_i}{dt} =  K_1^{ion}n_en_1 + K_2^{ion}n_en_2 - (K_1^{rec} + K_2^{rec})n_e^2n_i$$

In this case, one could simplify the system by dropping the ion density equation, but in we might want to track many different ionization states so $n_e \neq n_i$ in general.

Even though we have only 6 reactions, the number of terms (ignoring potentially simplifying groupings) in this system is 16. The CRM features in ReMKiT1D are meant to simplify the building of many related terms, such as these. 

**NOTE**: In this notebook we are not concerned with physical validity. The rate values and expressions are not relevant to any real physical process. We instead focus on how one would build up an arbitrary CRM. As such, everything is in arbitrary units.

### Species information in ReMKiT1D

Many built-in terms and models, including those generated with CRM term generators, require knowledge of the species they are being constructed for. This includes both basic information about the species (such as atomic mass and charge), as well as any associated variables and unique IDs.

**NOTE**: ReMKiT1D enforces that the electron species, with name "e" has the speciesID of 0. Other than that, the convention so far in ReMKiT1D has been that non-electron charged species get negative indices and neutral species get positive indices. 

Let's add the species present in the toy model we just made. Note that the mass and charge values are not used by the CRM data. Add the remaining variables below, making sure you associate the densities of the species.

In [ ]:
rk.addSpecies(name="e",speciesID=0,associatedVars=["ne"]) # ReMKiT1D will detect the electron species automatically and populate the mass and charge fields
#[YOUR CODE HERE]

As you can see, we've already associated the variables we'd like to use. They don't need to be added to the wrapper yet. Let us add them as before.

### Variables 

We have to add the four densities we're evolving. Initialize them using the provided values.

In [ ]:
ne = ni = np.ones(1)

n1 = 0.5*np.ones(1)

n2 = np.zeros(1)

# [YOUR CODE HERE]

Let's also add a dummy variable for the electron temperatue so we can demonstrate derived transitions further down. We also need to add the time variable, as always.

In [ ]:
Te = 5*np.ones(1)

# [YOUR CODE HERE]

### Constructing the CRM modelbound data object

We can now start working with the CRM modelbound data object. We use the `crm_support` module to simplify this process. 

In [ ]:
crmData = crm.ModelboundCRMData()

To add a transition with a constant rate and transition energy we use the `simpleTransition` function in `crm_support`. Let's use this to add the spontaneous emission reaction. Complete the call to `simpleTransition` by setting the `inState` and `outState` arguments.

In [ ]:
spontEmission = crm.simpleTransition(transitionEnergy=-10,transitionRate=0.3)

crmData.addTransition("spontEmission",transitionProperties=spontEmission)

To add a transition with one or more of the rates calculated using a derivation we use `derivedTransition`.

When using `derivedTransition`s we must supply at least one derivation rule - the rule for the transition rate itself. Momentum and energy rate derivations can also be supplied. If the energy rate is not supplied, it is calculated as `transitionRate*transitionEnergy`. We do not use the energy or momentum rates in terms in this example. but we will show how they can be accessed below. 

Let's add the ionization and recombination rates. To loosly mimic how these behave in reality, let's make them proportional to $T_e$ and $n_e^2/T_e$, respectively. We use the Node/treeDerivation approach. Complete the below derivations keeping the supplied proportionality constants.

In [ ]:
rk.addCustomDerivation("ion1",derivOptions=treeDerivation(0.2)) #[YOUR CODE HERE]
rk.addCustomDerivation("recomb1",derivOptions=treeDerivation(0.2)) #[YOUR CODE HERE]

rk.addCustomDerivation("ion2",derivOptions=treeDerivation(0.8)) #[YOUR CODE HERE]
rk.addCustomDerivation("recomb2",derivOptions=treeDerivation(0.8)) #[YOUR CODE HERE]

Complete the below transition definitions

In [ ]:
ion1 = crm.derivedTransition(inStates=[],outStates=[],transitionEnergy=12,ruleName="ion1",requiredVars=[""]) #[YOUR CODE HERE]

ion2 = crm.derivedTransition(inStates=[],outStates=[],transitionEnergy=2,ruleName="ion2",requiredVars=[""]) #[YOUR CODE HERE]

recomb1 = crm.derivedTransition(inStates=[],outStates=[],transitionEnergy=-12,ruleName="recomb1",requiredVars=["",""]) #[YOUR CODE HERE]

recomb2 = crm.derivedTransition(inStates=[],outStates=[],transitionEnergy=-2,ruleName="recomb2",requiredVars=["",""]) #[YOUR CODE HERE]

crmData.addTransition("ion1",ion1)
crmData.addTransition("ion2",ion2)
crmData.addTransition("recomb1",recomb1)
crmData.addTransition("recomb2",recomb2)

For excitation let's use the same proportionality as for ionization

In [ ]:
rk.addCustomDerivation("exc",derivOptions=treeDerivation(0.3)) #[YOUR CODE HERE]

exc = crm.derivedTransition(inStates=[],outStates=[],transitionEnergy=10,ruleName="exc",requiredVars=[""]) #[YOUR CODE HERE]

crmData.addTransition("exc",exc)

### Build a model using CRM data and term generators

We can now use a term generator to generate the standard CRM rate terms from the modelbound data. First we need to build a model and add the modelbound data to it using the dictionary form of it as the argument to `setModelboundData`. 

In [ ]:
#[YOUR CODE HERE]

The standard term generator for CRMs will take all reactions and generate matrix terms with diagonal stencils evolving the first associated variable (densities) of the input and output states. The implict variable is chosen to be the final variable in the `inStates` list, so for [0,1] the implicit variable will be "n1". 

Finer control over the generated terms is possible by selecting which implicit group the terms should be added to, which transitions should be included, as well as which states should be evolved. We use the default term generator, which will evolve all states using all transitions, and have them all in one term group.

Use `addTermGenerator` to add the below object to your crm model. Do not forget to add the model to the wrapper!

In [ ]:
crmTermGenerator = crm.termGeneratorCRM()

#[YOUR CODE HERE]

As you can see, your model has no terms, just a term generator.

### Extractor manipulator

We can directly extract variables from modelbound data using the `extractorManipulator` from `simple_containers`. Let's get the energy rate of the "recomb1" transition. This is the fourth added transition, and the energy moment is the second moment, so we will try to extract the modelbound variable `rate2index4` into the newly added variable `recombEn`.

Here we also come across manipulator priority. There are 5 possible priority levels, which each level including all those below it:

0. Manipulator is called on all internal integrator iterations (nonlinear BDE iterations in this case)
1. Manipulator is called at the end of each (internal) timestep (in any one integration step)
2. Manipulator is called at the end of each integration step (here there is only one integration step and no internal step timestep control so 1 and 2 would behave the same)
3. Manipulator is called at the end of each integration call (global timestep)
4. Manipulator is called before data writing to HDF5 file

Priority 4 is the default, and is in general used for diagnostic variables. At present, manipulators with priority > 4 are never called.

Add the new derived variable "recombEn", use `extractorManipulator` to specify the model, data name, and target variable name. You can add the manipulator to the wrapper using the wrapper's `addManipulator` function.

In [ ]:
#[YOUR CODE HERE]

### Time integration

In [ ]:
rk.addIntegrator("BE",sc.picardBDEIntegrator(nonlinTol=1e-12,absTol=10.0,convergenceVars=["ne","ni","n1","n2"])) 

rk.setIntegratorGlobalData(initialTimestep=0.0001) 

bdeStep = sc.IntegrationStep("BE")

for tag in rk.modelTags():
    bdeStep.addModel(tag)

rk.addIntegrationStep("StepBDE",bdeStep.dict())

rk.setFixedNumTimesteps(20000)
rk.setFixedStepOutput(400)

### Write config file

Remember to run ReMKiT1D with a single MPI process for this session.

In [ ]:
rk.writeConfigFile()

### Data analysis

In [ ]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=100,dpi=150)

numFiles=50
loadpath = rk.hdf5Filepath
loadFilenames = [loadpath+f'ReMKiT1DVarOutput_{i}.h5' for i in range(numFiles+1)]
loadedData = io.loadFromHDF5(rk.varCont, filepaths=loadFilenames)
loadedData

Let's look at the evolution of the variables

In [ ]:
pn.extension(comms="vscode") # change comms if not using VSCode
dashboard = ds.ReMKiT1DDashboard(loadedData,rk.grid)

dashboard.fluidMultiComparison(["ne","ni","n1","n2","recombEn"],fixedPosition=True)